In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras

## データ作り
- BPTT(Back Propagation Through Time)も考えて、window幅をとった入力を予め用意
    - 理想的には、時間全てをさかのぼってwを調整するのが良い
    - けれど、**勾配消失/爆発**があるので、t = [10,100]くらい(通常のRNNの場合)
    - `max_len`分のsliding windowにする

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
df = pd.read_csv("AirportPassengers.csv",delimiter=";").dropna()
data = []
target = []
max_len = 24
dim = 1
# 正規化
maximum = df.Passengers.max()
minimum = df.Passengers.min()
df["Passengers"] = (df.Passengers-minimum)/(maximum-minimum)
# データを箱に入れる
for i in range(len(df)-max_len-1):
    data.append(df.Passengers.values[i:i+max_len])
    target.append(df.Passengers.values[i+max_len+1])
# データの整形
data = np.array(data).reshape(len(data),max_len,dim)
target = np.array(target).reshape(-1,1)
# データの分割
from sklearn.model_selection import train_test_split
N_train = int(len(data)*0.7)
N_test = len(data) - N_train
X_train, X_validation, Y_train, Y_validation = train_test_split(data, target, test_size=N_test)


## KerasでRNN
構成


In [ ]:
from keras.models import Sequential
from keras.layers import  Dense
#一番シンプルなやつ
from keras.layers.recurrent import SimpleRNN
from keras.layers.recurrent import LSTM, GRU #改良版
model = Sequential()
# ネットワーク構成
input_shape=(max_len, dim)
# model.add(SimpleRNN(units=64, kernel_initializer="random_uniform",input_shape=input_shape))
#model.add(LSTM(units=64, input_shape=input_shape))
model.add(GRU(units=64, input_shape=input_shape))

model.add(Dense(input_shape[1],activation="linear"))
# optimizerの設定
from keras.optimizers import Adam
model.compile(loss="mse", optimizer=Adam())

## 学習と予測

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
epochs = 50
batch_size = 10
model.fit(X_train, Y_train,
          batch_size=batch_size, epochs=epochs,
          validation_data=(X_validation, Y_validation))
prediction = []
data_in = data.reshape(data.shape[0],data.shape[1])[N_train]

iteration = len(data) - N_train
for _ in range(iteration):
#     print(data_in)
    pred = model.predict(data_in.reshape(1,-1,1))
    data_in = np.delete(data_in, 0)
    data_in = np.hstack((data_in, pred[0]))
    prediction.append(pred[0,0])
passenger = list(df.Passengers)
data_num = len(passenger)
plt.plot(passenger,label="original")
plt.plot(range(N_train+max_len+1, N_train+max_len+iteration+1), prediction,label="prediction")
plt.legend()